# Amazon Bedrock AgentCore Runtime での Strands Agents を使用したストリーミングレスポンス

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用してストリーミングレスポンスを実装する方法を学びます。この例では、部分的な結果が利用可能になった時点でストリーミングする方法を示し、大量のコンテンツを生成したり、処理に時間がかかる操作に対してより応答性の高いユーザーエクスペリエンスを提供します。

### チュートリアルの詳細

|情報| 詳細|
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | ストリーミング機能付き会話型|
| エージェントタイプ | 単一 |
| エージェントフレームワーク | Strands Agents |
| LLM モデル | Anthropic Claude Sonnet 4 |
| チュートリアルコンポーネント | AgentCore Runtime でのストリーミングレスポンス、Strands Agent および Amazon Bedrock モデル |
| チュートリアルの分野 | 分野横断的 |
| 例の複雑さ | 簡単 |
| 使用 SDK | Amazon BedrockAgentCore Python SDK および boto3|

### チュートリアルのアーキテクチャ

このチュートリアルでは、ストリーミングエージェントを AgentCore ランタイムにデプロイする方法について説明します。

デモンストレーションのために、ストリーミング機能を備えた Amazon Bedrock モデルを使用する Strands Agent を使用します。

この例では、`get_weather` と `get_time` という 2 つのツールを持つシンプルなエージェントを使用しますが、ストリーミングレスポンス機能を備えています。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime 上のエージェントからのストリーミングレスポンス
* リアルタイムの部分的結果配信
* ストリーミング機能を持つ Amazon Bedrock モデルの使用
* 非同期ストリーミングサポートを備えた Strands Agents の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が稼働していること

In [1]:
#!uv add -r requirements.txt --active
import os
os.environ["AWS_PROFILE"] = "cline2"

# AgentCore Runtime へのストリーミングエージェントのデプロイ準備

ストリーミングエージェントを AgentCore Runtime にデプロイしましょう。ストリーミング機能は、エントリーポイント関数で非同期ジェネレーターまたは yield ステートメントを使用すると、AgentCore SDK によって自動的に処理されます。

ストリーミング実装のポイント：
* エントリーポイント関数には `async def` を使用する
* チャンクが利用可能になったら `yield` を使用してストリーミングする
* AgentCore SDK は Server-Sent Events (SSE) 形式を自動的に処理する
* クライアントは Content-Type: text/event-stream のレスポンスを受け取る

### Amazon Bedrock モデルとストリーミングを使用した Strands エージェント
Amazon Bedrock モデルを使用した Strands エージェントのストリーミング実装を見てみましょう。

In [2]:
%%writefile strands_claude_streaming.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# 計算機ツールをインポートする
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# カスタムツールの作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

@tool
def get_time():
    """ 現在時刻を取得する """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""あなたは役立つアシスタントです。簡単な計算ができ、天気を教え、現在の時刻を提供することができます。"""
)

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    ストリーミング機能を持つエージェントを呼び出す
    この関数は、非同期ジェネレーターを使用して AgentCore Runtime で
    ストリーミングレスポンスを実装する方法を示します
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    
    try:
        # 利用可能になった各チャンクをストリーミングする
        async for event in agent.stream_async(user_input):
            if "data" in event:
                yield event["data"]
            
    except Exception as e:
        # ストリーミングコンテキストでエラーを適切に処理する
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

Overwriting strands_claude_streaming.py


# AgentCore Runtime におけるストリーミングの理解

AgentCore Runtime でストリーミングを使用すると、いくつかのことが自動的に行われます：

### サーバー送信イベント (SSE) 形式
* AgentCore SDK は yield されたデータを自動的に SSE 形式に変換します
* 各 yield は SSE ストリームの中で `data: ` イベントになります
* Content-Type は自動的に `text/event-stream` に設定されます

### クライアント処理
* クライアントはエージェントがリクエストを処理する際にリアルタイムの更新を受け取ります
* これにより、段階的なレスポンス表示とより良いユーザーエクスペリエンスが可能になります
* クライアントは完全なレスポンスが準備される前に部分的な結果を処理できます

### エラー処理
* ストリーミングレスポンスには適切なエラー処理を含める必要があります
* エラーはストリームの一部として yield できます
* ストリームは関数が完了するか、未処理の例外が発生すると終了します

## AgentCore Runtime へのストリーミングエージェントのデプロイ

`CreateAgentRuntime` 操作は包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定することができます。また、プロトコル設定 (HTTP、MCP) や認証メカニズムを構成して、クライアントがエージェントと通信する方法を制御することもできます。

**注意:** 運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore ランタイムにデプロイします。

### ランタイムロールの作成

始める前に、AgentCore ランタイム用の IAM ロールを作成しましょう。これは、あなたのために事前に開発されたユーティリティ関数を使用して行います。

In [3]:
import sys
import os

# 現在のノートブックのディレクトリを取得する
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils.py の場所まで上に移動する
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加する
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="strands_claude_streaming3"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '455f4f85-5be9-4d89-8642-d706b0143ce0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 01 Aug 2025 06:42:46 GMT', 'x-amzn-requestid': '455f4f85-5be9-4d89-8642-d706b0143ce0', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-strands_claude_streaming3-role
recreating agentcore-strands_claude_streaming3-role
attaching role policy agentcore-strands_claude_streaming3-role


### AgentCore Runtime デプロイメントの構成

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および要件ファイルを含む AgentCore Runtime デプロイメントを構成します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを構成します。

構成ステップでは、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [4]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/strands_claude_streaming.py, bedrock_agentcore_name=strands_claude_streaming
Configuring BedrockAgentCore agent: strands_claude_streaming3


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/Dockerfile
Generated .dockerignore: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/.dockerignore
Keeping 'strands_claude_streaming3' as default agent
Bedrock AgentCore configured: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/Dockerfile'), dockerignore_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/03-advanced-concepts/01-streaming-agent-response/.dockerignore'), runtime='Docker', region='us-east-1', account_id='320462930492', execution_role='arn:aws:iam::320462930492:role/agentcore-strands_claude_streaming3-role', ecr_repository=None, auto_create_ecr=True)

### AgentCore Runtime へのストリーミングエージェントの起動

Docker ファイルができたので、ストリーミングエージェントを AgentCore Runtime に起動しましょう。これにより Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [5]:
launch_result = agentcore_runtime.launch()

Launching Bedrock AgentCore agent 'strands_claude_streaming3' to cloud
Docker image built: bedrock_agentcore-strands_claude_streaming3:latest
Using execution role from config: arn:aws:iam::320462930492:role/agentcore-strands_claude_streaming3-role
✅ Execution role validation passed: arn:aws:iam::320462930492:role/agentcore-strands_claude_streaming3-role
Uploading to ECR...
Getting or creating ECR repository for agent: strands_claude_streaming3
✅ ECR repository available: 320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3


✅ Reusing existing ECR repository: 320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3


Authenticating with registry...
Registry authentication successful
Tagging image: bedrock_agentcore-strands_claude_streaming3:latest -> 320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3:latest
Pushing image to registry...


The push refers to repository [320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3]
aa1ae2eeae7c: Preparing
90f47ebac8eb: Preparing
e19ff0bedb8e: Preparing
b69d75e1a664: Preparing
7c79870d7561: Preparing
ce892b8d3e89: Preparing
16cc4c7d5d83: Preparing
329cb79996ca: Preparing
e26286284f6e: Preparing
dd97e58b4e81: Preparing
ce892b8d3e89: Waiting
16cc4c7d5d83: Waiting
329cb79996ca: Waiting
e26286284f6e: Waiting
dd97e58b4e81: Waiting
e19ff0bedb8e: Layer already exists
7c79870d7561: Layer already exists
b69d75e1a664: Layer already exists
90f47ebac8eb: Layer already exists
ce892b8d3e89: Layer already exists
16cc4c7d5d83: Layer already exists
e26286284f6e: Layer already exists
329cb79996ca: Layer already exists
dd97e58b4e81: Layer already exists
aa1ae2eeae7c: Pushed


Image pushed successfully
Image uploaded to ECR: 320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3
Deploying to Bedrock AgentCore...


latest: digest: sha256:12ac582e9ea892a5ee579b2ba4d8abe5aa0a8cf4076617b9966a535996707cc0 size: 2415


✅ Agent created/updated: arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/strands_claude_streaming3-ECbdeg8Rpl
Polling for endpoint to be ready...
Agent endpoint: arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/strands_claude_streaming3-ECbdeg8Rpl/runtime-endpoint/DEFAULT
Deployed to cloud: arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/strands_claude_streaming3-ECbdeg8Rpl
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/strands_claude_streaming3-ECbdeg8Rpl-DEFAULT
   /aws/bedrock-agentcore/runtimes/strands_claude_streaming3-ECbdeg8Rpl-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude_streaming3-ECbdeg8Rpl-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude_streaming3-ECbdeg8Rpl-DEFAULT --since 1h


### AgentCore ランタイムのステータス確認
AgentCore ランタイムをデプロイしたので、そのデプロイ状況を確認しましょう

In [6]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_streaming3


'READY'

### AgentCore Runtime をストリーミングで呼び出す

最後に、ペイロードを使って AgentCore Runtime を呼び出し、ストリーミングレスポンスを受け取ることができます

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [7]:
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "what the weather is like?"
})
invoke_response

Invoking BedrockAgentCore agent 'strands_claude_streaming3' via cloud endpoint


{'response': '"I\'ll check the current"\n" weather for you."\n"The current"\n" weather is"\n" sunny!"\n" It"\n"\'s a nice day with clear,"\n" sunny"\n" conditions"\n"."'}

### AgentCore Runtime を boto3 でストリーミングで呼び出す

AgentCore Runtime が作成されたので、任意の AWS SDK で呼び出すことができます。ストリーミングレスポンスの場合は、Server-Sent Events 形式を処理する必要があります。

In [8]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# ストリーミングレスポンスの場合、EventStream を処理する必要があります
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# レスポンスがストリーミングかどうかを確認する
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("Processing streaming response with boto3:")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # "data: " プレフィックスを削除する
                print(f"Received streaming chunk: {data}")
                content.append(data.replace('"', ''))
    
    # ストリーミングされたレスポンスを完全に表示する
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # 非ストリーミング レスポンスの処理
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")

Processing streaming response with boto3:
Received streaming chunk: 2 
Received streaming chunk: + 1 = 3


2  + 1 = 3

# ストリーミングレスポンスの利点

ストリーミングレスポンスには以下のような主要な利点があります：

### ユーザーエクスペリエンス
* **即時フィードバック**：ユーザーは結果が利用可能になるとすぐに部分的な結果を見ることができます
* **体感パフォーマンス**：合計時間が同じであっても、レスポンスがより速く感じられます
* **段階的表示**：長いレスポンスを徐々に表示することができます

### 技術的メリット
* **メモリ効率**：すべてをメモリにロードせずに大きなレスポンスを処理できます
* **タイムアウト防止**：長時間実行される操作でのタイムアウトを回避できます
* **リアルタイム処理**：利用可能になったデータをリアルタイムで処理できます

### ユースケース
* **コンテンツ生成**：長文の文章、レポート、ドキュメント
* **データ分析**：複雑な計算からの段階的な結果
* **マルチステップワークフロー**：複雑なエージェントの推論過程の進捗を表示
* **リアルタイムモニタリング**：モニタリングエージェントからのライブアップデート

## クリーンアップ (任意)

作成した AgentCore Runtime をクリーンアップしましょう

In [9]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_streaming3',
 'strands_claude_streaming3-ECbdeg8Rpl',
 'bedrock-agentcore-strands_claude_streaming3')

In [10]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用したストリーミングエージェントの実装とデプロイに成功しました！

## 学んだこと：
* async ジェネレーターを使用したストリーミングレスポンスの実装方法
* AgentCore Runtime が SSE フォーマットを自動的に処理する方法
* クライアント側でストリーミングレスポンスを処理する方法
* ユーザーエクスペリエンスとパフォーマンスにおけるストリーミングの利点

## 次のステップ：
* ユースケースに合わせて異なるストリーミングパターンを試す
* 複雑なマルチステップワークフローのためのカスタムストリーミングロジックを実装する
* ストリーミングと Memory や Gateway などの他の AgentCore 機能を組み合わせる方法を探る
* より良い UX のためのクライアント側ストリーミング可視化の実装を検討する